In [ ]:
import cv2
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib as mpl
from matplotlib import cm
from matplotlib import colors
import networkx as nx
from scipy.interpolate import interp1d
from scipy import ndimage
from scipy.interpolate import splprep, splev
from scipy.signal import savgol_filter

#### 运动数据读取

In [ ]:
path = r'Y:\\SZX\\2025_wbi_analysis\\202509_TurnLabelCheck\\20250115_4.5g-24d-ov_08'
f_path = [f for f in os.listdir(path) if '_mot_vid_mid' in f][0]
df_mot_mid = pd.read_csv(os.path.join(path, f_path))

In [ ]:
df_mot_slice = df_mot_mid[['Time','X', 'Y',"x_velocity","y_velocity", 'angle_m','angle_md']]

## 前进-后退

### 1. 分割前进后退(+平滑)

In [ ]:
# 前进后退
# 根据阈值分割前进后退
threshold = 120
df_mot_slice.loc[:,'forward'] = 0
df_mot_slice.loc[df_mot_slice["angle_m"]>=threshold,'forward'] = 1

In [ ]:
# 平滑reversal
# 平滑
# 定义结构元素（卷积核）大小
close_size = 50  # 你可以调整这个值，通常为3或5
open_size = 50
# 先进行闭操作（先膨胀后腐蚀）：填充小的空洞
closed = ndimage.binary_closing(df_mot_slice['forward'].values, structure=np.ones(close_size))
# 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
opened = ndimage.binary_opening(closed, structure=np.ones(open_size))
# 将结果转换为整数并添加到 DataFrame
df_mot_slice['forward'] = opened.astype(int)

### 2. 求身体前进速度

#### 函数定义

In [ ]:
# 计算根据运动方向旋转后的朝头部方向运动向量
def rotation_mat_2(theta_degrees, mov_vec):
    mov_vec = np.array(mov_vec)
    if (theta_degrees == np.nan) or (theta_degrees == None):
        return None
    if (mov_vec is None) or (mov_vec.any()==np.nan):
        return None
    """
    统一的 2D 旋转矩阵
    θ > 0: 逆时针旋转
    θ < 0: 顺时针旋转
    """
    theta_rad = -np.radians(theta_degrees)
    cos_theta = np.cos(theta_rad)
    sin_theta = np.sin(theta_rad)
    rot_mat =  np.array([
        [cos_theta, -sin_theta],
        [sin_theta,  cos_theta]
    ])
    return np.dot(rot_mat, mov_vec)

rotation_mat_vec = np.vectorize(rotation_mat_2)
# 求向量投影
def project_vector_A_on_B(vector_A, vector_B):
    """
    返回C向量,方向与B相同,长度为A的投影
    """
    # 转换为numpy数组
    A = np.array(vector_A)
    B = np.array(vector_B)
    # 计算向量B的单位向量
    B_norm = np.linalg.norm(B)
    if B_norm == 0:
        print('B向量不能为零')
        return None
        # raise ValueError("向量B不能为零向量")
    B_unit = B / B_norm
    
    # 计算向量A在向量B上的投影长度
    projection_length = np.dot(A, B_unit)
    # 计算向量C = 投影长度 × 单位向量B
    vector_C = projection_length * B_unit
    return vector_C

#### 求投影在头部方向的运动向量

In [ ]:
# 运动方向向量
df_mot_slice.loc[:,'moving_vec'] = df_mot_slice.apply(lambda x: (x['x_velocity'], x['y_velocity']),axis = 1)

# 头部朝向向量
df_mot_slice.loc[:,'heading_vec'] = df_mot_slice.apply(lambda x: rotation_mat_2(x['angle_md'], x['moving_vec']), axis=1)
# 求投影头部朝向后的运动向量
df_mot_slice.loc[:,'head_moving'] = df_mot_slice.apply(lambda x: project_vector_A_on_B(x['moving_vec'], x['heading_vec']), axis=1)

## Omega turn

#### 读取中线数据

In [ ]:
# 读取中线
folder_path = r'Y:\\SZX\\2025_wbi_analysis\\202509_TurnLabelCheck\\20250115_4.5g-24d-ov_08'
npz_data = np.load(folder_path + '\output-0516.npz', allow_pickle=True)
data = npz_data['arr_0'].item()
data_keys = list(data.keys())
paths= {key: value['all_paths'] for key, value in data.items()}

# angle_md = np.array(list({key: value['angle_md'] for key, value in data.items()}.values()), dtype=float)
# angle_m = np.array(list({key: value['angle_m'] for key, value in data.items()}.values()), dtype=float)
# closest_idx = list({key: value['closest path'] for key, value in data.items()}.values())
# circular = list({key: value['circular'] for key, value in data.items()}.values())
# branching = list({key: value['branching'] for key, value in data.items()}.values())

In [ ]:
closest_idx = {key: value['closest path'] for key, value in data.items()}
pos_phr = {key: value['pos_phr'] for key, value in data.items()}
vector_h = {key: value['vector_h'] for key, value in data.items()} 
circular = {key: value['circular'] for key, value in data.items()}
branching = {key: value['branching'] for key, value in data.items()}

### 骨架合并和身体曲率计算

#### 函数定义

In [ ]:
def path_length(coords: np.ndarray) -> float:
    """polyline 长度（逐段欧氏距离累加）。coords: (N,2)"""
    if coords is None or len(coords) < 2:
        return 0.0
    d = np.diff(coords, axis=0)
    return float(np.sum(np.sqrt((d**2).sum(axis=1))))

def quantize_point(pt, q=1):
    """把像素点量化到网格上（容差聚类）。q=1 等价于原始整数像素匹配。"""
    return (int(round(pt[0]/q)*q), int(round(pt[1]/q)*q))

def cut_paths_to_edges(all_paths, q=1):
    """
    把多条骨架在“端点”和“交叉点”处切段，返回 MultiGraph：
    - 节点：关键点坐标（量化后）
    - 边：两关键点之间的子折线，数据里带 coords/length
    """
    # 1) 统计每个像素（量化后）被多少条路径/多少次命中，用于发现交叉
    occ = {}  # key -> list of (path_idx, local_idx)
    for pi, path in enumerate(all_paths):
        if path is None or len(path) == 0:
            continue
        for si, pt in enumerate(path):
            key = quantize_point(pt, q)
            occ.setdefault(key, []).append((pi, si))

    # 2) 关键点：每条 polyline 的首尾点 + 被多个 path 命中的点（交叉/重合点）
    keypoints = set()
    for pi, path in enumerate(all_paths):
        if path is None or len(path) == 0:
            continue
        keypoints.add(quantize_point(path[0], q))
        keypoints.add(quantize_point(path[-1], q))
    for key, hits in occ.items():
        # 命中数量>1，基本可视为交叉/共点（或一个 path 在该点有重复）
        if len(hits) > 1:
            keypoints.add(key)

    # 3) 在关键点处把每条 polyline 切段，生成边
    G = nx.MultiGraph()
    # 节点位置（用于方向判断）
    node_pos = {}  # key(node) -> 原始坐标（选第一次出现的真实像素）
    for kp in keypoints:
        # 选一个代表性坐标（量化格内实际像素），尽量从 occ 里取真实像素
        if kp in occ:
            pi, si = occ[kp][0]
            node_pos[kp] = tuple(map(int, all_paths[pi][si]))
        else:
            node_pos[kp] = (int(kp[0]), int(kp[1]))
        G.add_node(kp)

    for pi, path in enumerate(all_paths):
        if path is None or len(path) < 2:
            continue
        # 找到该 path 上属于关键点的索引
        cut_idx = []
        for si, pt in enumerate(path):
            if quantize_point(pt, q) in keypoints:
                cut_idx.append(si)
        # 确保首尾在里头
        if 0 not in cut_idx:
            cut_idx.insert(0, 0)
        if (len(path)-1) not in cut_idx:
            cut_idx.append(len(path)-1)
        # 去重并排序
        cut_idx = sorted(set(cut_idx))
        # 相邻关键点之间形成一条边
        for a, b in zip(cut_idx[:-1], cut_idx[1:]):
            if b <= a:
                continue
            sub = path[a:b+1]
            if len(sub) < 2:
                continue
            u = quantize_point(sub[0], q)
            v = quantize_point(sub[-1], q)
            w = path_length(sub)
            # 可能 u==v（零长度/回折），跳过
            if u == v or w == 0:
                continue
            # 入图，边上存 coords 和 length
            G.add_edge(u, v, length=w, coords=sub)

    # 把 node 真实坐标也记录上（便于后续方向判断）
    nx.set_node_attributes(G, {n: {'pos': node_pos[n]} for n in G.nodes})
    return G

def oriented_coords(edge_data, start_node_pos):
    """
    根据起点节点位置，返回边的坐标正向/反向（避免连接时倒序）。
    edge_data['coords'] 是边的折线；start_node_pos 是当前节点实际坐标
    """
    coords = edge_data['coords']
    if len(coords) == 0:
        return coords
    # 与起点更接近的一端作为开头
    d0 = np.linalg.norm(coords[0] - start_node_pos)
    d1 = np.linalg.norm(coords[-1] - start_node_pos)
    return coords if d0 <= d1 else coords[::-1]

def extract_non_branching_chains(G: nx.MultiGraph):
    """
    提取所有“非分叉链”：中间节点度数=2，端点为“度!=2 的节点”或到头。
    同时处理纯环（整个连通分量所有节点度数=2）的情况。
    返回：list of dict，每个 dict 包含 {'nodes': [...], 'coords': np.ndarray, 'length': float}
    """
    chains = []
    visited = set()  # 记录已走过的具体边 (u,v,key)（无向，按排序存）

    def mark(u,v,k):  # 无向规范化
        return (u, v, k) if u <= v else (v, u, k)

    deg = dict(G.degree())
    terminals = [n for n,d in deg.items() if d != 2]

    # —— 从端点出发的所有链 —— #
    for start in terminals:
        for _, nbr, key, data in G.edges(start, keys=True, data=True):
            m = mark(start, nbr, key)
            if m in visited:
                continue
            # 开始沿着非分叉链走
            path_nodes = [start]
            path_coords = []
            cur, prev = start, None

            while True:
                # 选择从 cur 出发且未访问的边
                next_edge = None
                for _, nb, k, d in G.edges(cur, keys=True, data=True):
                    mk = mark(cur, nb, k)
                    if mk in visited:
                        continue
                    # 避免立刻折返（如果有其它可选）
                    if prev is not None and nb == prev:
                        continue
                    next_edge = (cur, nb, k, d)
                    break
                if next_edge is None:
                    # 如果没有其它边，就看看能不能把“回退那条”也并上（首步情况）
                    for _, nb, k, d in G.edges(cur, keys=True, data=True):
                        mk = mark(cur, nb, k)
                        if mk not in visited:
                            next_edge = (cur, nb, k, d)
                            break
                if next_edge is None:
                    break

                u, v, k, d = next_edge
                # 方向化坐标并拼接（避免重复第一个点）
                start_pos = np.array(G.nodes[u]['pos'])
                seg = oriented_coords(d, start_pos)
                if len(path_coords) == 0:
                    path_coords = seg.copy()
                else:
                    path_coords = np.vstack([path_coords, seg[1:]])
                visited.add(mark(u, v, k))

                prev, cur = u, v
                path_nodes.append(cur)
                # 到达端点（度!=2）就停止
                if deg[cur] != 2:
                    break

            if len(path_coords) >= 2:
                chains.append({
                    'nodes': path_nodes,
                    'coords': path_coords,
                    'length': path_length(path_coords)
                })

    # —— 处理纯环：该连通分量所有节点度数都为2 —— #
    # 还有未访问的边说明它们属于环
    for u, v, k in list(G.edges(keys=True)):
        m = mark(u, v, k)
        if m in visited:
            continue
        # 从 u 出发沿环走到无法继续
        path_nodes = [u]
        path_coords = []
        cur, prev = u, None
        while True:
            next_edge = None
            for _, nb, kk, d in G.edges(cur, keys=True, data=True):
                mk = mark(cur, nb, kk)
                if mk in visited:
                    continue
                # 在环里第一次可以任意选边，之后避免直接折返
                if prev is not None and nb == prev:
                    continue
                next_edge = (cur, nb, kk, d)
                break
            if next_edge is None:
                # 没有未访问边了，环走完
                break
            a, b, kk, d = next_edge
            start_pos = np.array(G.nodes[a]['pos'])
            seg = oriented_coords(d, start_pos)
            if len(path_coords) == 0:
                path_coords = seg.copy()
            else:
                path_coords = np.vstack([path_coords, seg[1:]])
            visited.add(mark(a, b, kk))
            prev, cur = a, b
            path_nodes.append(cur)

        if len(path_coords) >= 2:
            chains.append({
                'nodes': path_nodes,
                'coords': path_coords,
                'length': path_length(path_coords)
            })

    return chains

def merge_and_find_longest_non_branching(all_paths, quant=1):
    """
    主入口：
    1) 在交叉/端点处切段并建图
    2) 提取所有非分叉链（含纯环）
    3) 返回最长链的坐标和长度
    """
    G = cut_paths_to_edges(all_paths, q=quant)
    chains = extract_non_branching_chains(G)
    if not chains:
        return None, 0.0, chains, G
    longest = max(chains, key=lambda c: c['length'])
    chain_dict = chains[0]
    all_coords = chain_dict['coords']
    # 返回所有轨迹
    # all_chains = list({key: value['coords'] for key, value in chain_dict.items()}.values())
    return longest['coords'], longest['length'], all_coords, G
def merge_and_find_non_branching(all_paths, quant=1):
    """
    主入口：
    1) 在交叉/端点处切段并建图
    2) 提取所有非分叉链（含纯环）
    3) 返回所有链的坐标
    """
    G = cut_paths_to_edges(all_paths, q=quant)
    chains = extract_non_branching_chains(G)
    if not chains:
        return None, G
    # chains是list，找一个元素，使得其在某个指标下最大，指标为元素长度
    # chain的每个元素是一条链的各个信息
    chains = sorted(chains, key=lambda c: c['length'], reverse=True)
    all_seq_path = [i['coords'] for i in chains]
    # 返回所有轨迹
    # all_chains = list({key: value['coords'] for key, value in chain_dict.items()}.values())
    return all_seq_path, G

In [ ]:
# 拆分相关函数
def path_length(path):
    """计算一条 path 的几何长度"""
    diffs = np.diff(path, axis=0)
    return np.sum(np.sqrt((diffs ** 2).sum(axis=1)))

def build_graph(paths, tol=1e-6):
    """把所有 paths 合并成一张图"""
    G = nx.Graph()
    node_id = {}

    def get_node_id(point):
        for nid, coord in node_id.items():
            if np.allclose(coord, point, atol=tol):
                return nid
        nid = len(node_id)
        node_id[nid] = point
        return nid

    for path in paths:
        src, dst = path[0], path[-1]
        src_id, dst_id = get_node_id(src), get_node_id(dst)
        length = path_length(path)
        G.add_edge(src_id, dst_id, weight=length, coords=path)

    return G, node_id

def longest_path(paths):
    """返回合并后图上的最长路径坐标"""
    G, node_id = build_graph(paths)
    degrees = dict(G.degree())
    endpoints = [n for n, d in degrees.items() if d == 1]

    longest_len = -1
    longest_coords = None

    for i in range(len(endpoints)):
        for j in range(i+1, len(endpoints)):
            u, v = endpoints[i], endpoints[j]
            for path_nodes in nx.all_simple_paths(G, u, v):
                coords = []
                total_len = 0
                for k in range(len(path_nodes)-1):
                    a, b = path_nodes[k], path_nodes[k+1]
                    edge = G[a][b]
                    coords_edge = edge["coords"]

                    # 判断方向
                    if np.allclose(coords_edge[0], node_id[a]):
                        coords.extend(coords_edge.tolist())
                    else:
                        coords.extend(coords_edge[::-1].tolist())

                    total_len += edge["weight"]

                if total_len > longest_len:
                    longest_len = total_len
                    longest_coords = np.array(coords)

    return longest_coords
def longest_path_old(paths):
    """返回合并后图上的最长路径坐标"""
    G, node_id = build_graph(paths)
    degrees = dict(G.degree())
    endpoints = [n for n, d in degrees.items() if d == 1]

    longest_len = -1
    longest_coords = None

    for i in range(len(endpoints)):
        for j in range(i+1, len(endpoints)):
            u, v = endpoints[i], endpoints[j]
            for path_nodes in nx.all_simple_paths(G, u, v):
                coords = []
                total_len = 0
                for k in range(len(path_nodes)-1):
                    edge = G[path_nodes[k]][path_nodes[k+1]]
                    coords.extend(edge["coords"].tolist())
                    total_len += edge["weight"]
                if total_len > longest_len:
                    longest_len = total_len
                    longest_coords = np.array(coords)

    return longest_coords


#### 1. 骨架拆分
seq_paths

In [ ]:
# 骨架拆分
seq_paths = {}
for key, value in paths.items():
    # print(merge_and_find_non_branching(value, quant=3))
    seq_path_i, G = merge_and_find_non_branching(
        value,
        quant=3   # 若不同段的交点不完全重合，可设 quant=2 或 3 像素，把近点聚成同一节点
    )
    seq_paths[key] = seq_path_i

#### 2. 骨架合并
longest_paths

In [ ]:
# 骨架合并，得到最长骨架
longest_paths = {}
for key, value in seq_paths.items():
    longest_coords = longest_path(value)
    longest_paths[key] = longest_coords

#### 3. 计算曲率(sum_dtheta)

##### 函数定义

In [ ]:
def resample_skeleton(skeleton, step=1.0):
    """
    把骨架按固定像素间距重采样
    skeleton: (N,2) 数组
    step: 每隔多少像素取一个点
    """
    skeleton = np.array(skeleton)
    if len(skeleton) < 3:
        return skeleton
    
    # 去掉重复点
    diffs = np.diff(skeleton, axis=0)
    mask = np.any(diffs != 0, axis=1)
    skeleton = skeleton[np.r_[True, mask]]
    if len(skeleton) < 3:
        return skeleton
    
    x, y = skeleton[:,0], skeleton[:,1]
    seglen = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    dist = np.concatenate(([0], np.cumsum(seglen)))   # 单位：像素
    total_len = dist[-1]
    
    # 归一化到 [0,1] 供 splprep 使用
    u = dist / total_len
    try:
        tck, _ = splprep([x, y], s=0, u=u)
    except Exception as e:
        print("splprep error:", e)
        return skeleton
    
    # 在 [0,total_len] 上等间距采样
    new_dist = np.arange(0, total_len, step)
    new_u = new_dist / total_len
    new_points = np.array(splev(new_u, tck)).T
    return new_points

def resample_skeleton_2(skeleton, step=1.0):
    # 等像素间距采样
    total_len = len(skeleton)
    if total_len <= step*2:
        return []
    idx_series = np.arange(0,total_len, step)
    selected_points = skeleton[idx_series,:]
    return selected_points

def curvature_from_angles(skeleton, step=1.0):
    """
    通过角度差计算骨架曲率
    返回：
        curvatures: 每个点的角度差 (带正负号)
        mean_curvature: 曲率强度指标
    """
    pts = resample_skeleton_2(skeleton, step=step)
    if not len(pts):
        return np.nan

    diffs = np.diff(pts, axis=0)
    angles = np.arctan2(diffs[:,1], diffs[:,0])
    # 解开角度防止跳变
    angles_unwrapped = np.unwrap(angles)
    # 相邻角度差
    dtheta = np.diff(angles_unwrapped)
    # # 曲率指标（你可以换成 np.mean(np.abs(dtheta))）
    # mean_curvature = np.sqrt(np.mean(dtheta**2))
    return np.degrees(dtheta)

In [ ]:
# 等间距重采样和滑动平均

def moving_average(data, window_size=5):
    """对二维数据做滑动平均"""
    kernel = np.ones(window_size) / window_size
    x = np.convolve(data[:,0], kernel, mode='same')
    y = np.convolve(data[:,1], kernel, mode='same')
    return np.vstack((x,y)).T

def resample_and_smooth(points, window_size=5):
    # --- 等弧长重采样 ---
    x, y = points[:,0], points[:,1]
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    s = np.concatenate(([0], np.cumsum(dist)))
    total_length = s[-1]
    s_uniform = np.linspace(0, total_length, len(points))
    fx = interp1d(s, x, kind='linear')
    fy = interp1d(s, y, kind='linear')
    resampled = np.vstack((fx(s_uniform), fy(s_uniform))).T
    
    # --- 平滑 ---
    smoothed = moving_average(resampled, window_size)
    return resampled, smoothed
def remove_outliers_by_jump(points, max_step=10):
    """
    去掉瞬间跨度过大的点
    points: (N,2) array
    max_step: 相邻点最大允许移动距离 (像素)
    """
    x, y = points[:,0], points[:,1]
    dist = np.sqrt(np.diff(x)**2 + np.diff(y)**2)
    # 第一个点保留
    mask = np.ones(len(points), dtype=bool)
    
    # 如果跨度大于阈值，就把后一帧去掉
    mask[1:][dist > max_step] = False
    
    return points[mask]
def resample_and_dthatas(long_paths, step):
    # 采样和平均
    resampled = {}
    # smoothed = {}
    for key, skelon in long_paths.items():
        if type(skelon) == type(None):
            resampled[key] =  []
        else:
            # 去掉瞬移离群点
            skelon = remove_outliers_by_jump(skelon, max_step=10)
            resampled[key],_ = resample_and_smooth(skelon, window_size=50)
    dtheta = {}
    for key, skelon in resampled.items():
        if type(None)==type(skelon):
            dtheta[key] = None
        elif (len(skelon)) < 200:
            dtheta[key] = None
        else:
            dtheta_i = curvature_from_angles(skelon, step=step)
            sum_dtheta_i = round(np.sum(dtheta_i),2)
            dtheta[key]={'dtheta':dtheta_i,'sum_dtheta':sum_dtheta_i}
    return resampled, dtheta


##### 重采样后计算曲率(用于分turn不用确定头部朝向)
sum_dtheta

In [ ]:
resampled, dtheta = resample_and_dthatas(longest_paths,step=20)
sum_dtheta = {
    key: (value['sum_dtheta'] if isinstance(value, dict) and 'sum_dtheta' in value else None)
    for key, value in dtheta.items()
    }
# # 曲率计算结果生成df
df_sum_dtheta = pd.Series(sum_dtheta, name='sum_dtheta')
print(f'df_sum_dtheta的长度{len(df_sum_dtheta)}和最大index{df_sum_dtheta.index.max()}')

### 路径比例计算(path_ratio, path_len)

##### 函数定义

In [ ]:
# 求路径比例和前两条路径长度的df
def path_ratio(paths):
    ratios = {}
    len_dict = {}
    for key, value in paths.items():
        if len(value) >= 2:
            # 如果大于2，求前两个的比例
            len_ls = [len(path) for path in value]
            len_ls.sort(reverse=True)
            ratio = len_ls[0]/len_ls[1]
            lens = len_ls[:2]
        else:
            ratio = np.nan
            lens = []
        ratios[key] = ratio
        len_dict[key] = lens
    df_ratio = pd.Series(ratios, name='path_ratio').to_frame()
    df_len = pd.Series(len_dict, name='path_len').to_frame()
    df = df_ratio.join(df_len, how='left')

    return df

In [ ]:
# 求1的开始和结束点
def get_turn_interval(df, col_name):
    # labels打印矩形
    labels = df[col_name].values
    n = len(labels)
    # 找出连续的 label==1 区间
    turn_intervals = []
    in_turn = False
    start = 0

    for i in range(n):
        if labels[i] == 1 and not in_turn:
            # turn开始
            start = i
            in_turn = True
        elif labels[i] != 1 and in_turn:
            # turn结束
            end = i - 1
            turn_intervals.append((start, end))
            in_turn = False

    # 如果最后一个点也是turn状态
    if in_turn:
        turn_intervals.append((start, n - 1))
    return turn_intervals

### 条件判断分turn

#### 合并分类数据完成条件判断

In [ ]:
# 加入曲率角度数据到运动数据（切片）
# df_mot_slice = df_mot_mid[['Time','X', 'Y', 'forward']]
df_mot_slice = df_mot_slice.join(df_sum_dtheta, how='left')
# 生成路径比例，加入路径比例信息信息数据到运动数据
df_path_ratio = path_ratio(paths)
df_mot_slice = df_mot_slice.join(df_path_ratio, how='left')

df_cir_branching = pd.DataFrame.from_dict(
    {"circular": circular, "branching": branching},
    orient="index"
).T
# 加入原骨架分析信息到运动数据
# df_mot_slice.drop(columns=['circular', 'branching'], inplace=True)
df_mot_slice = df_mot_slice.join(df_cir_branching, how='left')

生成一个新的df，标记turn和reversal_turn

+ “turn_pc”: 所有coling
+ “forward_subs_turn”: 不和omega turn重合的forward
+ "turn_cor": 所有Omega turn

In [ ]:
# 初始化 turn 列
df_mot_slice['turn'] = 0  
df = df_mot_slice.copy()
# 限制第二长的轨迹长度要大于200
df['path_lim'] = df['path_len'].apply(
    lambda x: 1 if (isinstance(x, (list, tuple, np.ndarray)) and len(x) > 1 and x[1] >= 200) else 0
)

df.loc[:,'turn'] = 0
df.loc[
    (df['circular'] == 1) |
    (df['branching'] == 1) |
    ((df['path_ratio'] <= 1.5)&(df['path_lim'] == 1)) |
    (df['sum_dtheta'].abs() >= 220)
    ,
    'turn'
] = 1

In [ ]:
# 平滑转向
# 定义结构元素（卷积核）大小
close_size = 50  # 你可以调整这个值，通常为3或5
open_size = 50
# 先进行闭操作（先膨胀后腐蚀）：填充小的空洞
closed = ndimage.binary_closing(df['turn'].values, structure=np.ones(close_size))
# 再进行开操作（先腐蚀后膨胀）：去除小的噪声点
opened = ndimage.binary_opening(closed, structure=np.ones(open_size))
# 将结果转换为整数并添加到 DataFrame
df['turn_pc'] = opened.astype(int)

In [ ]:
df['turn_color'] = df['turn_pc'].map({1:'red', 0:'blue'})

#### Reversal Turn标记

删除那些后接reversal的turn

In [ ]:
# 首先根据turn将reversal切断，只保留不为turn的reversal段
df['forward_subs_turn'] = df['forward']
df.loc[df['turn_pc']==1,'forward_subs_turn'] = 0
df['turn_cor']  = df['turn_pc'].copy()
# 计算所有reversal段的长度 ，如果大于threshold,看前面是否有turn，有的话将这个turn删除
threshold = 50
n = len(df)
# 找到连续的 reversal 段
# df["rev_group"] = (df["forward_subs_turn"].diff(1).ne(0)).cumsum() * df["forward_subs_turn"]
df['turn_group'] = (df["turn_cor"].diff().ne(0)).cumsum() * df["turn_cor"]
# 遍历每个 reversal 区间
for gid, grp in df.groupby('turn_group'):
    if gid == 0:
        continue

    # turn 段的结束位置（label）与位置索引（整数位置）
    end_label = grp.index[-1]
    end_pos = df.index.get_loc(end_label)

    # 寻找紧接着出现 reversal 的位置（允许 small gap）
    start_search_pos = end_pos + 1
    if start_search_pos >= n:
        # 如果数据到头了，舍弃
        continue
    found_pos = None

    for offset in range(threshold + 1):
        p = start_search_pos + offset
        if p >= n:
            break
        if df['forward_subs_turn'].iat[p] == 1:
            found_pos = p
            break
    if found_pos is None:
        # 没有紧接的 reversal
        continue

    # 计数 reversal 连续长度
    q = found_pos
    while q + 1 < n and df['forward_subs_turn'].iat[q + 1] == 1:
        # 遍历reversal的长度，当超过reversal时停
        q += 1
    rev_len = q - found_pos + 1

    if rev_len >= threshold:
        # 仅把这个 turn 段（grp.index）置 0
        df.loc[grp.index, "turn_cor"] = 0
    else:
        df.iloc[found_pos:found_pos+rev_len, df.columns.get_loc('forward_subs_turn')] = 0
        # 如果这个长度小于阈值，则仍然保留turn，并且把接着的这个reverse变为前进
df = df.drop(columns=["turn_group"])

#### 合并turn分类后结果到df中

In [ ]:
df_mot_slice[['turn_pc','turn_cor', "forward_subs_turn"]] = df[['turn_pc','turn_cor','forward_subs_turn']]

#### 验证

##### 可视化验证Reversal turn标记

In [ ]:
# 可视化验证
reversal_ints = get_turn_interval(df, 'forward')
subs_reversal_ints = get_turn_interval(df, 'forward_subs_turn')

# Turns: 开闭后的turn
# turn_ints = get_turn_interval(df, 'turn')
turn_pc_ints = get_turn_interval(df, 'turn_pc')
turn_cor_ints = get_turn_interval(df, 'turn_cor')
# artificial labeling
label_ints = get_turn_interval(df, 'label')
# 绘图
fig,ax = plt.subplots(4,1,figsize=(20,10), sharex=True)

# 绘制人工打标记
for start, end in subs_reversal_ints:
    ax[0].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
for start, end in turn_pc_ints:
    ax[0].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[0].set_title('automated labeling of turnclose:{close_size},open:{open_size}')

for start, end in subs_reversal_ints:
    ax[1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
for start, end in label_ints:
    ax[1].axvspan(start, end, color='red', alpha=1)  # alpha控制透明度
ax[1].set_title('artificial labeling of turn')

for start, end in subs_reversal_ints:
    ax[2].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
for start, end in turn_cor_ints:
    ax[2].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[2].set_title(f'corrected turn labeling')

for start, end in subs_reversal_ints:
    ax[3].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
ax[3].set_title(f'reversal')
desired_ticks = np.arange(0, len(df), 2000) # 生成一个数组 [10, 30, 50, 70, 90]
ax[3].set_xticks(desired_ticks)
plt.show()
# plt.xlim([2000,5000])

##### 画轨迹图验证

In [ ]:
# series = np.arange(0,len(df), 4000)
# for i in range(len(series)-1):
#     df_cut = df.iloc[series[i]:series[i+1],:]
#     fig, ax = plt.subplots(1,2,figsize=(20,8))
#     ax[0].scatter(df_cut['X'], df_cut['Y'], s=1, c = df_cut['turn_color'])
#     f2 = ax[1].scatter(df_cut['X'], df_cut['Y'], s=1, c = np.arange(series[i],series[i+1],1), cmap='bwr')
#     plt.colorbar(f2)
#     plt.suptitle(f'idx{series[i]}-{series[i+1]}')
#     plt.show()


##### 导入人工标注可视化验证

In [ ]:
# 导入label数据
path_dir = r'Y:\\SZX\\2025_wbi_analysis\\202509_TurnLabelCheck\\20250115_4.5g-24d-ov_08'
omega_file = '\\omega_labels.npy'
omega_np = np.load(path_dir+omega_file, allow_pickle=True)
df['label'] = np.array(omega_np)

In [ ]:
# 可视化验证
reversal_ints = get_turn_interval(df, 'forward')
turn_ints = get_turn_interval(df, 'turn')
# 开闭后的turn
turn_pc_ints = get_turn_interval(df, 'turn_pc')
# circular
cir_ints = get_turn_interval(df, 'circular')
# branching
bra_ints = get_turn_interval(df, 'branching')
# path_ratio
df['path_ratio_dis'] = np.where(df['path_ratio'] <= 1.5, 1, 0)
ratio_ints = get_turn_interval(df,'path_ratio_dis')
# sum_dtheta
df['sum_dtheta_dis'] = np.where(df['sum_dtheta'].abs() >= 220, 1, 0)
dtheta_ints = get_turn_interval(df, "sum_dtheta_dis")
# artificial labeling
label_ints = get_turn_interval(df, 'label')

# 绘图
fig,ax = plt.subplots(7,1,figsize=(20,10), sharex=True)

# 绘制人工打标记

for start, end in turn_ints:
    ax[0].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[0].set_title('automated labeling of turn')
for start, end in reversal_ints:
    ax[1].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
for start, end in label_ints:
    ax[1].axvspan(start, end, color='red', alpha=1)  # alpha控制透明度
ax[1].set_title('artificial labeling of turn')
for start, end in cir_ints:
    ax[6].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[6].set_title('circular')
for start, end in bra_ints:
    ax[3].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[3].set_title('branching')
for start, end in ratio_ints:
    ax[4].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[4].set_title('path_ratio')
for start, end in dtheta_ints:
    ax[5].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[5].set_title('sum_dtheta')
desired_ticks = np.arange(0, len(df), 2000) # 生成一个数组 [10, 30, 50, 70, 90]
ax[6].set_xticks(desired_ticks)
for start, end in reversal_ints:
    ax[2].axvspan(start, end, color='blue', alpha=1)  # alpha控制透明度
for start, end in turn_pc_ints:
    ax[2].axvspan(start, end, color='orange', alpha=1)  # alpha控制透明度
ax[2].set_title(f'turn_open_closed. close:{close_size},open:{open_size}')

plt.show()

##### 视频验证turn

In [ ]:
WHITE = (255, 255, 255)
YELLOW = (0, 255, 255)
CYAN = (255, 255, 0)
LIME_GREEN = (0, 255, 0)
PINK = (147, 20, 255)
ORANGE = (0, 165, 255)
RED = (0, 0, 255)
GOLD = (0, 215, 255)
colors = [(0,0,255), (0,255,0),(255,0,0)]
colors_2 = [(176, 106, 28),   # 钴蓝色
(51, 51, 204),    # 科学红
(0, 204, 102),    # 激光绿
(0, 255, 255),    # 荧光黄
(255, 0, 255),    # 洋红
(0, 77, 255)]

In [ ]:
df = df
# 骨架信息
series_skeleton = pd.Series(paths)        # 原始骨架
long_skeleton = pd.Series(longest_paths)
# 读取视频
cap = cv2.VideoCapture(os.path.join(folder_path, "c1_new_onnx.mp4"))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频总帧数:", total_frames)

# 参数
check_inv = 2
output_path = os.path.join(folder_path, "c1_midline_compare.mp4")
start_frame = 16800
delay = 30
frame_idx_end = total_frames
step  = 2
rotation_threshold = 220
label_col = 'turn_pc' 
cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

for frame_idx in range(start_frame, total_frames, step):
    # 视频定位当前帧
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)  # 定位到 frame_idx
    ret, frame = cap.read()
    if frame is None:
        continue
    if not ret:
        break
    
    # ================= 左图（原视频 + 自动标注（原始骨架） =================
    frame_left = frame.copy()
    cv2.putText(frame_left, f"Frame: {frame_idx}", (10, 500),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

    try:
        idx = data_keys.index(frame_idx)
        row = df.iloc[frame_idx]
        points_raw = series_skeleton.iloc[idx]
    except:
        pass
        
    for i, points in enumerate(points_raw):
        # 轨迹长度
        len_i = len(points) # 轨迹长度
        cv2.putText(frame_left, f"skel{i}:{len_i}", (350, 30+(i*20)),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, colors[i % 3], 2)
        # 轨迹
        pts = normalize_points(points) if len(points) > 0 else []
        for (x, y) in pts:
            if np.isnan(x) or np.isnan(y):
                continue
            cv2.circle(frame_left, (int(y), int(x)), 5, colors[i % 3], -1)
    
    # 字符打印
    # 判断turn
    if row[label_col]:
        cv2.putText(frame_left, label_col, (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    else:
        cv2.putText(frame_left, f"turn", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 200), 2)
    # 环形或交叉
    if row.circular:
        cv2.putText(frame_left, f"circle", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    else:
        cv2.putText(frame_left, f"circle", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (200, 200, 200), 2)
        
    if row.branching:
        cv2.putText(frame_left, f"branching", (80, 50),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), 2)
    else:
        cv2.putText(frame_left, f"branching", (80, 50),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (200, 200, 200), 2)
    if row.path_ratio<=3:
        cv2.putText(frame_left, f"ratio:{round(row.path_ratio,2)}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), 2)
    else:
        cv2.putText(frame_left, f"ratio:{round(row.path_ratio,2)}", (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (200, 200, 200), 2)
    if np.abs(row.sum_dtheta)>=rotation_threshold:
        cv2.putText(frame_left, f"rotation:{row.sum_dtheta}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (0, 0, 255), 2)  
    else:    
        cv2.putText(frame_left, f"rotation:{row.sum_dtheta}", (10, 90),
                cv2.FONT_HERSHEY_SIMPLEX,0.75, (200, 200, 200), 2)  
        
    # ================= 右图（原视频 + 合并骨架） =================
    frame_right = frame.copy()

    # 打印最长骨架（只有一条）
    try:
        idx = data_keys.index(frame_idx)
        row = df.iloc[frame_idx]
        # 拆分骨架
        points_long = long_skeleton.iloc[idx]
        if points_long is None:
            points_long = []
    except:
        points_long = []

    # 轨迹
    pts = normalize_points(points_long) if len(points_long) > 0 else []
    for (x, y) in pts:
        if np.isnan(x) or np.isnan(y):
            continue
        if row.label == 1:
            cv2.circle(frame_right, (int(y), int(x)), 5, (0,0,255), -1)
        else: 
            cv2.circle(frame_right, (int(y), int(x)), 5, (200,200,200), -1)
    # 打印label标记
    if row.label == 1:
        cv2.putText(frame_right, f"label", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,1, YELLOW, 2)
        
    # ================= 拼接左右视频 =================
    combined = np.hstack((frame_left, frame_right))

    cv2.imshow("Skeleton Compare", combined)

    key = cv2.waitKey(delay) & 0xFF
    if key == ord('p'):
        break
    if key == ord('q'):
        break
    elif key == ord(' '):
        cv2.waitKey(0)


cap.release()
cv2.destroyAllWindows()


## 头部曲率和角速度
这一部依赖于omega turn阶段合并后的骨架longest

### 1. 选择骨架(closest和longest二选一后根据咽喉位置选择头朝向)

In [ ]:
# 提取原最接近咽喉的骨架
closest_path = {}
for key, all_paths in paths.items():   # paths 是 dict 的情况
    idx = closest_idx[key]
    closest_path[key] = all_paths[idx]

#### 选择长骨架
sel_paths

In [ ]:
# 判断longest path的坐标头尾方向是否和closest坐标正确
# 根据长度判断是选择longest_path还是closest_path
sel_paths = {}
for key, longest in longest_paths.items():
    closest = closest_path[key]
    if type(longest)==type(None):
        # 重合并无骨架，选择closest
        sel_paths[key] = closest_path[key]
        continue
    # 计算头尾距离
    head = longest[0]
    tail = longest[-1]
    closest_head = closest[0]
    dist_head = (head[0]-closest_head[0])**2 + (head[1]-closest_head[1])**2
    dist_tail = (tail[0]-closest_head[0])**2 + (tail[1]-closest_head[1])**2
    if len(longest) > len(closest) & (min(dist_tail, dist_head) <= 10):
        # 两条轨迹的头是一样的，否则即使longest更长也不要
        sel_paths[key] = longest
    else:
        sel_paths[key] = closest

#### 确定头朝向
rev_sel_path 

In [ ]:
# 定义函数根据咽喉位置决定翻转
def process_skeleton(points, pos_phr, n_segments=30):
    """
    points: list of (x, y)
    pos_phr: (x, y) 咽喉点
    n_segments: 要分的段数
    """
    # 1. 转成 DataFrame
    points = np.array(points)
    points = resample_skeleton(points,step=1)
    n_points = len(points)

    # 2. 找最近的点
    pos_phr = np.array(pos_phr)
    dist = np.linalg.norm(points - pos_phr, axis=1)
    idx_phr = np.argmin(dist)
    # 4. 比较长度
    # len_left = idx_phr
    # len_right = len(dist)-idx_phr
    # 3. 计算左侧累计长度
    len_left = np.sum(
        np.linalg.norm(points[1:idx_phr+1] - points[:idx_phr], axis=1)
    )
    # 4. 计算右侧累计长度
    len_right = np.sum(
        np.linalg.norm(points[idx_phr+1:] - points[idx_phr:-1], axis=1)
    )

    # 5. 是否翻转
    if len_left > len_right:
        points = points[::-1]
    return points

In [ ]:
# 根据咽喉坐标,距离咽喉近的设为头
rev_sel_path = {}
for key, sel in sel_paths.items():
    pos_phr_i = pos_phr[key]
    re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
    rev_sel = process_skeleton(sel, re_pos_phr_i)
    rev_sel_path[key] = rev_sel

##### 其它：根据角度(舍弃)

In [ ]:
# def cosine_similarity_np(a, b):
#     """
#     使用NumPy计算两个向量的余弦相似度
#     """
#     # 计算点积 (dot product)
#     dot_product = np.dot(a, b)
#     # 计算各自的L2范数 (模)
#     norm_a = np.linalg.norm(a)
#     norm_b = np.linalg.norm(b)
#     # 计算余弦相似度
#     cosine_sim = dot_product / (norm_a * norm_b)
#     return cosine_sim
# # 根据头部向量方向决定翻转
# def process_skeleton_2(points, pos_phr,vector_h, interval = 100, k=None):
#     """
#     points: list of (x, y)
#     pos_phr: (x, y) 咽喉点
#     n_segments: 要分的段数
#     """
#     # 1. 转成 DataFrame
#     points = np.array(points)
#     # 重采样
#     points = resample_skeleton(points,step=1)
#     n_points = len(points)

#     # 2. 找最近的点
#     pos_phr = np.array(pos_phr)
#     dist = np.linalg.norm(points - pos_phr, axis=1)
#     idx_phr = np.argmin(dist)
#     idx_prev = max(idx_phr - interval, 0)
#     idx_next = min(idx_phr + interval, n_points - 1)
#     head = points[idx_prev]
#     tail = points[idx_next]
#     # 尾部指向头部
#     tail_head = np.array([head[0]-tail[0], head[1]-tail[1]])
#     head_tail = np.array([tail[0]-head[0], tail[1]-head[1]])
    
#     cos_head = cosine_similarity_np(tail_head, vector_h)
#     cos_tail = cosine_similarity_np(head_tail, vector_h)
    
#     # 5. 是否翻转
#     if cos_tail > cos_head:
#         rev_points = points[::-1]

#         if k % 1000 == 0:
#             plt.figure()
#             plt.scatter(points[:,0], points[:,1], s = 1, c = np.arange(len(points)), cmap='bwr')
#             # 打印向量
#             print(vector_h)
#             plt.quiver(pos_phr[0], pos_phr[1], 20*vector_h[0], 20*vector_h[1], angles='xy', scale_units='xy', scale=0.5, color='b')
#             plt.scatter(pos_phr[0], pos_phr[1], s = 20, color = 'blue')
#             plt.scatter(points[idx_phr][0], points[idx_phr][1], s = 20, color = 'g', label="nearest")
#             plt.scatter(points[idx_prev][0], points[idx_prev][1], s = 20, color = 'orange', label = 'pre')
#             plt.scatter(points[idx_next][0], points[idx_next][1], s = 20, color = 'r', label = 'post')
#             plt.legend()
#             plt.title(k)
#             plt.show()

#         return rev_points
    
#     else:
#         return points

### 2. 骨架分割+曲率计算
根据选择的骨架坐标(第一个是头部)。为骨架分段，约13个像素一段分割后验证
利用分段后的骨架计算头部向量以及头部曲率

#### 函数定义

In [ ]:
def calc_angle(vector1, vector2):
    """
        计算带符号的角度（-180° 到 180°）
        正角度表示从 vector1 到 vector2 是逆时针旋转
        负角度表示顺时针旋转
        """
    dot_product = np.dot(vector1, vector2)
    cross_product = np.cross(vector1, vector2)  # 在 2D 中，这给出标量值
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    denominator = magnitude1 * magnitude2
    if denominator < 1e-10:  # 设置一个很小的阈值
        # 处理零向量的情况
        cos_theta = 1.0 if np.allclose(vector1, vector2) else np.nan
        sin_theta = 0.0
    else:
        cos_theta = dot_product / (magnitude1 * magnitude2)
        sin_theta = cross_product / (magnitude1 * magnitude2)
    # 使用 arctan2 获取带符号的角度
    angle_rad = np.arctan2(sin_theta, cos_theta)
    return np.degrees(angle_rad)
def label_segments(points, step=30):
    df_bb = pd.DataFrame(points, columns=['X', 'Y'])
    seg_len = np.linalg.norm(points[1:] - points[:-1], axis=1)
    # 累计距离
    cumdist = np.concatenate([[0], np.cumsum(seg_len)])
    # 分段编号
    seg_id = (cumdist // step).astype(int)
    df_bb = df_bb.copy()
    df_bb['seg_id'] = seg_id
    colors_bgr = [
    (255, 0, 0),     # 蓝色
    (0, 255, 0),     # 绿色
    (0, 0, 255),     # 红色
    (0, 255, 255),   # 黄色 (BGR: cyan+red)
    (255, 0, 255),   # 洋红 (magenta)
    (255, 255, 0)    # 青色 (cyan)
    ]
    # 建立 seg_id 到颜色的映射
    dict_colors = {key: colors_bgr[key % len(colors_bgr)] for key in range(len(colors_bgr))}
    df_bb['color'] = df_bb['seg_id'].apply(lambda x: dict_colors[x%6])
    return df_bb

In [ ]:
def label_segments_curvature(points,pos_phr, step=30):
    if not len(points):
        return None, None, None
    df_bb = pd.DataFrame(points, columns=['X', 'Y'])
    seg_len = np.linalg.norm(points[1:] - points[:-1], axis=1)
    # 累计距离
    cumdist = np.concatenate([[0], np.cumsum(seg_len)])
    # 分段编号
    seg_id = (cumdist // step).astype(int)
    df_bb = df_bb.copy()
    df_bb['seg_id'] = seg_id
    # colors_bgr = [
    # (255, 0, 0),     # 蓝色
    # (0, 255, 0),     # 绿色
    # (0, 0, 255),     # 红色
    # (0, 255, 255),   # 黄色 (BGR: cyan+red)
    # (255, 0, 255),   # 洋红 (magenta)
    # (255, 255, 0)    # 青色 (cyan)
    # ]
    # # 建立 seg_id 到颜色的映射
    # dict_colors = {key: colors_bgr[key % len(colors_bgr)] for key in range(len(colors_bgr))}
    # df_bb['color'] = df_bb['seg_id'].apply(lambda x: dict_colors[x%6])

    # 分段后根据咽喉所在段求头部曲率
    pos_phr = np.array([pos_phr[0],pos_phr[1]])
    dist = np.linalg.norm(points - pos_phr, axis=1)
    idx_phr = np.argmin(dist)
    df_phr = df_bb.iloc[idx_phr]
    seg_phr_idx = df_bb.iloc[idx_phr]['seg_id']
    # 提取靠后的点
    # seg_pos_idx = (seg_phr_idx-2)
    df_seg_pos = df_bb[df_bb['seg_id']==(seg_phr_idx+6)]
    # 头部顶点
    df_start = df_bb.iloc[0]
    if not len(df_seg_pos):
        curvature = None
        pos_phr = None
        phr_node = None
    else:
        df_pos_end = df_seg_pos.iloc[-1]
        # 指向向量
        pos_phr = [df_phr['X']-df_pos_end['X'], df_phr['Y']-df_pos_end['Y']]
        phr_node = [df_start['X']-df_phr['X'], df_start['Y']-df_phr['Y']]
        curvature = calc_angle(pos_phr, phr_node)

    # 根据前两段求头部指向向量用于计算角速度
    df_seg2 = df_bb[df_bb['seg_id']==2]
    if not len(df_seg2):
        head_vector = [np.nan, np.nan]
    else:
        seg2_end = df_seg2.iloc[-1]
        head_vector = [df_start['X']-seg2_end['X'], df_start['Y']-seg2_end['Y']]

    return df_bb, curvature, head_vector, pos_phr, phr_node

#### 计算头部曲率，头部向量

In [46]:
curvature = {}
head_vector = {}
post_phr = {}
phr_noses = {}
for key, sel in rev_sel_path.items():
    pos_phr_i = pos_phr[key]
    re_pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
    df_bb_i, curvature_i, head_vector_i, post_phr_i, phr_noses_i = label_segments_curvature(sel,re_pos_phr_i, step=13)
    curvature[key] = curvature_i
    head_vector[key] = head_vector_i
    post_phr[key] = post_phr_i
    phr_noses[key] = phr_noses_i

In [47]:
# 将曲率和头部向量加入df中
# df_sum_dtheta = pd.Series(sum_dtheta, name='sum_dtheta')
df_cur = pd.Series(curvature, name='curvature')
df_head_vec = pd.Series(head_vector, name='head_vector')
df_sel = pd.Series(rev_sel_path, name='sel_paths')

In [48]:
df_mot_slice['curvature'] = df_cur
df_mot_slice['head_vector'] = df_head_vec
df_mot_slice['sel_paths'] = df_sel

#### 计算头部角速度

##### 函数定义

In [49]:
def cal_head_agl_velocity(df, window_size=300, frame_rate=38):
    '''
    对向量的角度变化进行滤波计算
    '''
    # 平滑计算角速度
    head_vectors =  df['head_vector']
    head_vec_filled = head_vectors.apply(lambda v: v if isinstance(v, (list, np.ndarray)) else [np.nan, np.nan])
    head_arr = np.vstack(head_vec_filled)
    dy = head_arr[:,0]
    dx = head_arr[:,1]
    # 假设 dx, dy shape (T,)
    angles = np.arctan2(dy,dx)          # radians
    angles_interp = pd.Series(angles).interpolate(limit_direction="both").to_numpy()
    angles_unwrapped = np.unwrap(angles_interp)

    # 采样间隔 dt (s) —— 例：300帧 = 15s -> dt = 15/300 = 0.05
    dt = 1 / frame_rate  # = 0.05

    angles_unwrapped = np.array(angles_unwrapped, dtype=float)
    mask = np.isfinite(angles_unwrapped)
    angles_clean = angles_unwrapped[mask]

    # 2. 设置合适的窗口
    window = min(window_size, len(angles_clean) // 2 * 2 + 1)  # 不超过数据长度的最大奇数
    if window < 3: 
        window = 3  # 最小窗口

    # 3. 滤波
    angular_velocity = savgol_filter(
        angles_clean,
        window_length=window,
        polyorder=1,
        deriv=1,
        delta=dt,
        mode="interp"
    )
    # 若需要度/秒：
    angular_velocity_deg = np.degrees(angular_velocity)
    df['ang_velocity'] = angular_velocity_deg
    return df


##### 头部角速度

In [50]:
df_mot_slice = cal_head_agl_velocity(df_mot_slice)

#### 视频验证

In [ ]:
sel_ls_paths = {}
for key, value in rev_sel_path.items():
    sel_ls_paths[key] = [value]

##### 视频验证头部角速度

In [ ]:
# 从灰色到红色的颜色条
def create_gray_to_red_colormap():
    """生成从灰色到红色的 256 色查找表 (BGR 格式)"""
    lut = np.zeros((256, 1, 3), dtype=np.uint8)

    for i in range(256):
        # 归一化到 [0,1]
        t = i / 255.0  
        # 从灰色 (128,128,128) -> 红色 (255,0,0)
        gray = int(128 * (1 - t) + 255 * t)  # 红通道逐渐变强
        g = int(128 * (1 - t))               # 绿通道逐渐变弱
        b = int(128 * (1 - t))               # 蓝通道逐渐变弱

        lut[i, 0] = (b, g, gray)  # BGR

    return lut
def value_to_color(value, vmin=-180, vmax=180):
    """单个数值 -> BGR 颜色"""
    # 映射到 [0,255]
    normed = int(np.clip((value - vmin) / (vmax - vmin) * 255, 0, 255))
    lut = create_gray_to_red_colormap()
    return tuple(int(c) for c in lut[normed, 0])  # (B,G,R)

In [ ]:
# 读取视频
cap = cv2.VideoCapture(os.path.join(folder_path, "c1_new_onnx.mp4"))
series_skeleton = pd.Series(sel_ls_paths)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频总帧数:", total_frames)
# 计算头部朝向时的切割像素长度
body_l = 200

# 检查间隔（每隔n帧）
check_inv = 10
delay = 20
output_path = os.path.join(folder_path, "c1_midline.mp4")  # 导出视频路径
frame_idx = 0
# 结束帧
frame_idx_end = total_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
error_ls = []

while cap.isOpened() and frame_idx <= frame_idx_end:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % check_inv != 0:
        frame_idx += 1
        continue
    # 显示帧索引
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 画出向量
    color_vector_h = (128, 0, 128)  # 紫色
    color_vector_m = (0, 165, 255)    # 明黄色
    color_vector_mh= (216, 191, 216) # 淡紫色
    if frame_idx in data_keys:
        idx = frame_idx   # 找到对应帧的列表索引
    else:
        print('不存在这一帧:frame',frame_idx)
        frame_idx += 1
        continue
    # 读取咽喉点坐标
    pos_phr_i = np.array(pos_phr[idx], dtype=float)
    # 读取两个向量和角速度
    vector_h_i= np.array(vector_h[idx], dtype=float)
    curvature_i = curvature[idx]
    head_vector_i = head_vector[idx]

    # row = df.iloc[idx]
    # cur_i = row.curvature
    # if cur_i:
    #     cv2.putText(frame, f"cur_i: {cur_i}", (10, 60), 
    #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # agl_vel = row.ang_velocity
    # if agl_vel:
    #     color_agl = value_to_color(agl_vel)
    #     cv2.putText(frame, f"agl_vel: {agl_vel:.2f}", (10, 90), 
    #                 cv2.FONT_HERSHEY_SIMPLEX, 1, color_agl, 2)
    
    if frame_idx < max(data_keys):
        if frame_idx in data_keys:
            idx = frame_idx   # 找到对应帧的列表索引
            points_raw = series_skeleton[idx]  # 原始骨架
        else:
            print('不存在骨架：frame',frame_idx)
            frame_idx += 1
            continue
        # 有多条骨架，分别画出
        colors = [(0,0,255), (0,255,0),(255,0,0)]
        i = 0
        for points in points_raw:
            midline_points = normalize_points(points)  # 返回 (N,2) int32
            df_bb = label_segments(midline_points, step=30)

            j = 0
            for _,row in df_bb.iterrows():
                # 打印第一帧
                if j == 0:
                    cv2.circle(frame, (row.Y, row.X), 10, (255,0,255), thickness=-1)
                cv2.circle(frame, (row.Y, row.X), 3, row.color, -1)
                j += 1

            df_seg2 = df_bb[df_bb['seg_id']==2]
            if not len(df_seg2):
                pass
            else:
                seg2_end = df_seg2.iloc[-1]
                # 头部向量打印
                if type(head_vector_i)==type(None):
                    pass
                else:
                    norm_head = np.linalg.norm(head_vector_i)
                    vector_head_i = head_vector_i/norm_head
                    # 计算起始结束点位置
                    length_factor = 40
                    start_point_h = (int(seg2_end[1]), int(seg2_end[0]))
                    end_point_h = (int(seg2_end[1] + vector_head_i[1]*length_factor), int(seg2_end[0] + vector_head_i[0]*length_factor))
                    cv2.line(frame, start_point_h, end_point_h, (0,200,255), 15)

            i += 1
    
    # ----------------- 绘制右侧矢量图 -----------------
    
    panel_size = 512  # 右侧面板大小
    vector_scale = 80 # 画向量长度缩放
    panel = np.ones((panel_size, frame.shape[0], 3), dtype=np.uint8) * 255  # 高度与视频一致
    center = (panel_size // 2, frame.shape[0] // 2)

    df = df_mot_slice.copy()
    row = df.iloc[idx]
    # cur_i = row.curvature
    # if cur_i:
    #     cv2.putText(frame, f"cur_i: {cur_i}", (10, 60), 
    #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    agl_vel = row.ang_velocity
    if agl_vel:
        color_agl = value_to_color(agl_vel)
        cv2.putText(panel, f"agl_vel: {agl_vel:.2f}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color_agl, 2)

    if head_vector_i is not None and not np.any(np.isnan(head_vector_i)):
        norm_head = np.linalg.norm(head_vector_i)
        if norm_head > 0:
            vec = (head_vector_i / norm_head) * vector_scale
            end_point = (int(center[0] + vec[1]), int(center[1] + vec[0]))  # y向下
            cv2.arrowedLine(panel, center, end_point, (0,0,255), 3, tipLength=0.3)
    cv2.putText(panel, "head_vector", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1)
    
    # 将原视频帧和右侧面板合并
    combined_frame = np.hstack([frame, panel])
    cv2.imshow("Skeleton Check", combined_frame)
    key = cv2.waitKey(delay) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


In [ ]:
# 读取视频
# cap = cv2.VideoCapture(os.path.join(folder_path, "c1_onnx.mp4"))
cap = cv2.VideoCapture(os.path.join(folder_path, "c1_new_onnx.mp4"))
series_skeleton = pd.Series(sel_ls_paths)
# 储存曲率和角速度信息的df
df = df_mot_slice.copy()
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频总帧数:", total_frames)
# 计算头部朝向时的切割像素长度
body_l = 200
# 检查间隔（每隔n帧）
check_inv =25
delay = 30
output_path = os.path.join(folder_path, "c1_midline.mp4")  # 导出视频路径
frame_idx = 0
# 结束帧
frame_idx_end = total_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)



error_ls = []

while cap.isOpened() and frame_idx <= frame_idx_end:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % check_inv != 0:
        frame_idx += 1
        continue
    # 显示帧索引
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 画出向量
    color_vector_h = (128, 0, 128)  # 紫色
    color_vector_m = (0, 165, 255)    # 明黄色
    color_vector_mh= (216, 191, 216) # 淡紫色
    if frame_idx in data_keys:
        idx = frame_idx   # 找到对应帧的列表索引
    else:
        print('不存在这一帧：frame',frame_idx)
        frame_idx += 1
        continue
    # 读取咽喉点坐标
    pos_phr_i = np.array(pos_phr[idx], dtype=float)
    # 读取两个向量和角速度
    vector_h_i= np.array(vector_h[idx], dtype=float)
    # 打印头部方向向量
    # vec_h_print = [round(vector_h_i[0], 3), round(vector_h_i[1], 3)]
    # cv2.putText(frame, f"vector_h: {vec_h_print}", (10, 60), 
    #             cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 打印曲率和角速度
    row = df.iloc[idx]
    cur_i = row.curvature
    if cur_i:
        cv2.putText(frame, f"cur_i: {cur_i}", (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    agl_vel = row.ang_velocity
    if agl_vel:
        color_agl = value_to_color(agl_vel)
        cv2.putText(frame, f"agl_vel: {agl_vel:.2f}", (10, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, color_agl, 2)
    # 向量打印
    if  np.isnan(vector_h_i).any():
        print('无方向向量，不打印')
        pass
    else:

        norm_h = np.linalg.norm(vector_h_i)
        vector_h_i = vector_h_i/norm_h

        # 计算起始结束点位置
        length_factor = body_l

        start_point_h = (int(pos_phr_i[0] - vector_h_i[0]*length_factor), int(pos_phr_i[1] - vector_h_i[1]*length_factor))
        end_point_h = (int(pos_phr_i[0] + vector_h_i[0]*length_factor), int(pos_phr_i[1] + vector_h_i[1]*length_factor))
        
        
        # 需要强制转换为int才能打印
        pos_phr_i = (int(pos_phr_i[0]), int(pos_phr_i[1]))
        start_point_h = (int(start_point_h[0]), int(start_point_h[1]))
        end_point_h = (int(end_point_h[0]), int(end_point_h[1]))
        cv2.line(frame, start_point_h, end_point_h, color_vector_h, 5)
        cv2.circle(frame, pos_phr_i, 10, color_vector_h, thickness=-1)

    if frame_idx < max(data_keys):
        if frame_idx in data_keys:
            idx = frame_idx   # 找到对应帧的列表索引
            points_raw = series_skeleton[idx]  # 原始骨架
        else:
            print('不存在骨架：frame',frame_idx)
            frame_idx += 1
            continue
        # 有多条骨架，分别画出
        colors = [(0,0,255), (0,255,0),(255,0,0)]
        i = 0
        for points in points_raw:
            midline_points = normalize_points(points)  # 返回 (N,2) int32
            j = 0
            for (x, y) in midline_points:
                # 打印第一帧
                if j == 0:
                    cv2.circle(frame, (y,x), 10, (255,0,255), thickness=-1)
                cv2.circle(frame, (y,x), 3, colors[i%3], -1)
                j += 1
            i += 1

    cv2.imshow("Skeleton Check", frame)
    key = cv2.waitKey(delay) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


##### 视频验证骨架分割和选择计算区域(曲率计算)

In [ ]:
def normalize_points(points):
    """确保骨架点是 (N,2) int32 数组"""
    arr = np.array(points, dtype=np.float32)
    arr = np.vstack(arr)                # 去掉长度为1的维度

    arr = arr.astype(np.int32)           # OpenCV 要 int32
    return arr


In [ ]:
# 读取视频
cap = cv2.VideoCapture(os.path.join(folder_path, "c1_onnx.mp4"))
series_skeleton = pd.Series(sel_ls_paths)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频总帧数:", total_frames)
# 计算头部朝向时的切割像素长度
body_l = 200

# 检查间隔（每隔n帧）
check_inv = 10
delay = 20
output_path = os.path.join(folder_path, "c1_midline.mp4")  # 导出视频路径
frame_idx = 14500
# 结束帧
frame_idx_end = total_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
error_ls = []

while cap.isOpened() and frame_idx <= frame_idx_end:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % check_inv != 0:
        frame_idx += 1
        continue
    # 显示帧索引
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 画出向量
    color_vector_h = (128, 0, 128)  # 紫色
    color_vector_m = (0, 165, 255)    # 明黄色
    color_vector_mh= (216, 191, 216) # 淡紫色
    if frame_idx in data_keys:
        idx = frame_idx   # 找到对应帧的列表索引
    else:
        print('不存在这一帧:frame',frame_idx)
        frame_idx += 1
        continue
    # 读取咽喉点坐标
    pos_phr_i = np.array(pos_phr[idx], dtype=float)
    # 读取两个向量和角速度
    vector_h_i= np.array(vector_h[idx], dtype=float)
    curvature_i = curvature[idx]
    head_vector_i = head_vector[idx]
    # 读取计算曲率的向量
    post_phr_i = post_phr[idx]
    phr_noses_i = phr_noses[idx]

    # 打印曲率
    if type(curvature_i)!=type(None):
        cv2.putText(frame, f"Curvature: {round(curvature_i,2)}", (10, 100), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 打印头部方向向量
    vec_h_print = [round(vector_h_i[0], 3), round(vector_h_i[1], 3)]

    cv2.putText(frame, f"vector_h: {vec_h_print}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 向量打印
    if  np.isnan(vector_h_i).any():
        print('无方向向量，不打印')
        pass
    else:
        norm_h = np.linalg.norm(vector_h_i)
        vector_h_i = vector_h_i/norm_h
        # 计算起始结束点位置
        length_factor = body_l
        start_point_h = (int(pos_phr_i[0] - vector_h_i[0]*length_factor), int(pos_phr_i[1] - vector_h_i[1]*length_factor))
        end_point_h = (int(pos_phr_i[0] + vector_h_i[0]*length_factor), int(pos_phr_i[1] + vector_h_i[1]*length_factor))

        # 需要强制转换为int才能打印
        pos_phr_i = (int(pos_phr_i[0]), int(pos_phr_i[1]))
        start_point_h = (int(start_point_h[0]), int(start_point_h[1]))
        end_point_h = (int(end_point_h[0]), int(end_point_h[1]))
        cv2.line(frame, start_point_h, end_point_h, color_vector_h, 5)
        cv2.circle(frame, pos_phr_i, 10, color_vector_h, thickness=-1)
    
    if frame_idx < max(data_keys):
        if frame_idx in data_keys:
            idx = frame_idx   # 找到对应帧的列表索引
            points_raw = series_skeleton[idx]  # 原始骨架
        else:
            print('不存在骨架：frame',frame_idx)
            frame_idx += 1
            continue
        # 有多条骨架，分别画出
        colors = [(0,0,255), (0,255,0),(255,0,0)]
        i = 0
        for points in points_raw:
            midline_points = normalize_points(points)  # 返回 (N,2) int32
            df_bb = label_segments(midline_points, step=30)

            j = 0
            for _,row in df_bb.iterrows():
                # 打印第一帧
                if j == 0:
                    cv2.circle(frame, (row.Y, row.X), 10, (255,0,255), thickness=-1)
                cv2.circle(frame, (row.Y, row.X), 3, row.color, -1)
                j += 1
            
        

            df_seg2 = df_bb[df_bb['seg_id']==2]
            if not len(df_seg2):
                pass
            else:
                seg2_end = df_seg2.iloc[-1]
                # 头部向量打印
                if type(head_vector_i)==type(None):
                    pass
                else:
                    norm_head = np.linalg.norm(head_vector_i)
                    vector_head_i = head_vector_i/norm_head
                    # 计算起始结束点位置
                    length_factor = 40
                    start_point_h = (int(seg2_end[1]), int(seg2_end[0]))
                    end_point_h = (int(seg2_end[1] + vector_head_i[1]*length_factor), int(seg2_end[0] + vector_head_i[0]*length_factor))
                    cv2.line(frame, start_point_h, end_point_h, (0,200,255), 15)

            i += 1

        # 打印曲率向量

        if (type(post_phr_i)==type(None)) or (type(phr_noses_i)==type(None)) :
                pass
        else:
            norm_pos = np.linalg.norm(post_phr_i)
            norm_phr = np.linalg.norm(phr_noses_i)
            vector_pos = post_phr_i/norm_pos
            vector_phr = phr_noses_i/norm_phr

            pos_phr_i = np.array(pos_phr[idx], dtype=float)
            # 计算起始结束点位置
            length_factor = 60
            start_point_pos = (int(pos_phr_i[0]), int(pos_phr_i[1]))
            end_point_pos = (int(pos_phr_i[0] - vector_pos[1]*length_factor), int(pos_phr_i[1] - vector_pos[0]*length_factor))
            cv2.line(frame, start_point_pos, end_point_pos, (0,0,255), 15)

            start_point_phr = (int(pos_phr_i[0]), int(pos_phr_i[1]))
            end_x = pos_phr_i[0] + vector_phr[1]*length_factor
            end_y = pos_phr_i[1] + vector_phr[0]*length_factor
            if np.isnan(end_x) or np.isnan(end_y):
                pass
            else:
                end_point_phr = (int(end_x),int(end_y) )
                cv2.line(frame, start_point_phr, end_point_phr, (255,0,0), 15)


    cv2.imshow("Skeleton Check", frame)
    key = cv2.waitKey(delay) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


##### 视频验证头部方向选择

In [ ]:
def normalize_points(points):
    """确保骨架点是 (N,2) int32 数组"""
    arr = np.array(points, dtype=np.float32)
    arr = np.vstack(arr)                # 去掉长度为1的维度

    arr = arr.astype(np.int32)           # OpenCV 要 int32
    return arr

In [ ]:
# 读取视频
cap = cv2.VideoCapture(os.path.join(folder_path, "c1_new_onnx.mp4"))
# cap = cv2.VideoCapture(os.path.join(folder_path, "c1_onnx.mp4"))
series_skeleton = pd.Series(sel_ls_paths)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print("视频总帧数:", total_frames)
# 计算头部朝向时的切割像素长度
body_l = 200

# 检查间隔（每隔n帧）
check_inv = 10
delay = 20
output_path = os.path.join(folder_path, "c1_midline.mp4")  # 导出视频路径
frame_idx = 0
# 结束帧
frame_idx_end = total_frames
cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)



error_ls = []

while cap.isOpened() and frame_idx <= frame_idx_end:
    ret, frame = cap.read()
    if not ret:
        break
    if frame_idx % check_inv != 0:
        frame_idx += 1
        continue
    # 显示帧索引
    cv2.putText(frame, f"Frame: {frame_idx}", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 画出向量
    color_vector_h = (128, 0, 128)  # 紫色
    color_vector_m = (0, 165, 255)    # 明黄色
    color_vector_mh= (216, 191, 216) # 淡紫色
    if frame_idx in data_keys:
        idx = frame_idx   # 找到对应帧的列表索引
    else:
        print('不存在这一帧：frame',frame_idx)
        frame_idx += 1
        continue
    # 读取咽喉点坐标
    pos_phr_i = np.array(pos_phr[idx], dtype=float)
    # 读取两个向量和角速度
    vector_h_i= np.array(vector_h[idx], dtype=float)
    # 打印头部方向向量
    vec_h_print = [round(vector_h_i[0], 3), round(vector_h_i[1], 3)]
    cv2.putText(frame, f"vector_h: {vec_h_print}", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # 向量打印
    if  np.isnan(vector_h_i).any():
        print('无方向向量，不打印')
        pass
    else:

        norm_h = np.linalg.norm(vector_h_i)
        vector_h_i = vector_h_i/norm_h

        # 计算起始结束点位置
        length_factor = body_l

        start_point_h = (int(pos_phr_i[0] - vector_h_i[0]*length_factor), int(pos_phr_i[1] - vector_h_i[1]*length_factor))
        end_point_h = (int(pos_phr_i[0] + vector_h_i[0]*length_factor), int(pos_phr_i[1] + vector_h_i[1]*length_factor))
        
        
        # 需要强制转换为int才能打印
        pos_phr_i = (int(pos_phr_i[0]), int(pos_phr_i[1]))
        start_point_h = (int(start_point_h[0]), int(start_point_h[1]))
        end_point_h = (int(end_point_h[0]), int(end_point_h[1]))
        cv2.line(frame, start_point_h, end_point_h, color_vector_h, 5)
        cv2.circle(frame, pos_phr_i, 10, color_vector_h, thickness=-1)

    if frame_idx < max(data_keys):
        if frame_idx in data_keys:
            idx = frame_idx   # 找到对应帧的列表索引
            points_raw = series_skeleton[idx]  # 原始骨架
        else:
            print('不存在骨架：frame',frame_idx)
            frame_idx += 1
            continue
        # 有多条骨架，分别画出
        colors = [(0,0,255), (0,255,0),(255,0,0)]
        i = 0
        for points in points_raw:
            midline_points = normalize_points(points)  # 返回 (N,2) int32
            j = 0
            for (x, y) in midline_points:
                # 打印第一帧
                if j == 0:
                    cv2.circle(frame, (y,x), 10, (255,0,255), thickness=-1)
                cv2.circle(frame, (y,x), 3, colors[i%3], -1)
                j += 1
            i += 1

    cv2.imshow("Skeleton Check", frame)
    key = cv2.waitKey(delay) & 0xFF   # 播放速度 (30ms/帧)，也可以改大
    if key == ord('q'):   # 按 q 退出
        break
    elif key == ord(' '): # 按空格暂停
        cv2.waitKey(0)

    frame_idx += 1

cap.release()
cv2.destroyAllWindows()


##### 验证骨架选择和头部方向选择

In [ ]:
# 打印验证骨架方向以及长度是否选择了最长的
keys = np.arange(23720,24000,10)
for i,k in enumerate(keys):
    fig, ax = plt.subplots(1,4,figsize = (20,6))
    # 第一张图打印所有骨架，最近骨架打印红色
    all_paths_i = paths[k]
    closest_i = closest_path[k]
    pos_phr_i = pos_phr[k]
    pos_phr_i = [pos_phr_i[1], pos_phr_i[0]]
    for p in all_paths_i:
        ax[0].scatter(p[:,0], p[:,1], s = 0.8, color = 'grey')
    ax[0].scatter(closest_i[:,0], closest_i[:,1], s=0.8, color='red')
    ax[0].scatter(closest_i[0,0], closest_i[0,1], s=20, color='red')
    ax[0].set_title('all paths(closest in red)')
    ax[0].scatter(pos_phr_i[0], pos_phr_i[1], s= 20, color='blue')

    # 第二张图
    longest_i = longest_paths[k]
    path_series = np.arange(0,len(longest_i))
    ax[1].scatter(longest_i[:,0], longest_i[:,1], s=0.8, c= path_series, cmap='bwr')
    ax[1].scatter(longest_i[0,0], longest_i[0,1], s=20, color='red')
    ax[1].set_title('longest backbone')
    ax[1].scatter(pos_phr_i[0], pos_phr_i[1], s= 20, color='blue')
    # 第三张图打印select骨架，颜色渐变
    sel = sel_paths[k]
    path_series = np.arange(0,len(sel))
    ax[2].scatter(sel[:,0], sel[:,1], s=0.8, c= path_series, cmap='bwr')
    ax[2].scatter(sel[0,0], sel[0,1], s=20, color='red')
    ax[2].scatter(pos_phr_i[0], pos_phr_i[1], s= 20, color='blue')
    ax[2].set_title('Selected backbone')
    # 第四张图打印翻转的骨架
    sel_i = rev_sel_path[k]
    path_series = np.arange(0,len(sel_i))
    ax[3].scatter(sel_i[:,0], sel_i[:,1], s=0.8, c= path_series, cmap='bwr')
    ax[3].scatter(sel_i[0,0], sel_i[0,1], s=20, color='red')
    ax[3].scatter(pos_phr_i[0], pos_phr_i[1], s= 20, color='blue')
    ax[3].set_title('Rev Selected backbone')
    plt.suptitle(k)
    plt.show()

## 写出

In [51]:
df_mot_slice.columns

Index(['Time', 'X', 'Y', 'x_velocity', 'y_velocity', 'angle_m', 'angle_md',
       'forward', 'moving_vec', 'heading_vec', 'head_moving', 'sum_dtheta',
       'path_ratio', 'path_len', 'circular', 'branching', 'turn', 'turn_pc',
       'turn_cor', 'forward_subs_turn', 'curvature', 'head_vector',
       'sel_paths', 'ang_velocity'],
      dtype='object')

In [52]:
df_append = df_mot_slice[['forward', 'moving_vec', 'heading_vec', 'head_moving', 'sum_dtheta',
                          'path_ratio', 'path_len', 'circular', 'branching', 'turn', 'turn_pc','turn_cor', 
                          'forward_subs_turn', 'curvature', 'head_vector','sel_paths', 'ang_velocity']]

In [56]:
if 'circular' in df_mot_mid.columns:
    df_mot_mid.drop(columns=['circular'], inplace=True)
if 'branching' in df_mot_mid.columns:
    df_mot_mid.drop(columns=['branching'], inplace=True)

In [57]:
df_mot_out = df_mot_mid.join(df_append, how='left')

In [58]:
f_output = '_mot_vid_mid_turn.csv'
df_mot_out.to_csv(os.path.join(path, f_output))